In [1]:
import pandas as pd
import numpy as np
drkg_file = '../data/drkg/drkg.tsv'
df = pd.read_csv(drkg_file, sep="\t", header=None)
import pickle
import os
import rdkit
from rdkit import Chem
from rdkit.Chem import DataStructs
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import rdMolHash
from tqdm import tqdm

RDKit WARNING: [14:36:06] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
df

,0,1,2
0,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::2157
1,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::5264
2,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::2158
3,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::3309
4,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::28912
...,...,...,...
5874256,Gene::29099,STRING::REACTION::Gene:Gene,Gene::1643
5874257,Gene::51645,STRING::REACTION::Gene:Gene,Gene::3183
5874258,Gene::865,STRING::CATALYSIS::Gene:Gene,Gene::983
5874259,Gene::1066,STRING::BINDING::Gene:Gene,Gene::7365


In [3]:
all_nodes = np.unique(np.concatenate((df[0].values, df[2].values)))

In [4]:
len(all_nodes)

97238

In [5]:
string = df[df[1].apply(lambda s: s[:6] == 'STRING')]

In [6]:
all_string_nodes = np.unique(np.concatenate((string[0].values, string[2].values)))

In [7]:
all_string_nodes

array(['Gene::1', 'Gene::10', 'Gene::100', ..., 'Gene::9993',
       'Gene::9994', 'Gene::9997'], dtype=object)

In [8]:
string_gene_ids = [s[6:] for s in all_string_nodes]

In [9]:
len(string_gene_ids)

18316

In [10]:
string_gene_ids[:20]

['1',
 '10',
 '100',
 '1000',
 '10000',
 '100008586',
 '10001',
 '10002',
 '10003',
 '100037417',
 '100038246',
 '10004',
 '100049587',
 '10005',
 '10006',
 '10007',
 '10008',
 '10009',
 '1001',
 '10010']

## Comparing with landmark genes

In [11]:
# Get list of landmark genes
gene_info = pd.read_csv('../Data/L1000_PhaseI/GSE92742_Broad_LINCS/GSE92742_Broad_LINCS_gene_info.txt', sep="\t")
landmark_gene_list = list(gene_info[gene_info['pr_is_lm'] == 1]["pr_gene_id"].astype(str))

In [12]:
len(landmark_gene_list)

978

In [13]:
len(set(landmark_gene_list).intersection(set(string_gene_ids)))

972

## Drug ids

In [14]:
df[1].unique()

array(['bioarx::HumGenHumGen:Gene:Gene', 'bioarx::VirGenHumGen:Gene:Gene',
       'bioarx::DrugVirGen:Compound:Gene',
       'bioarx::DrugHumGen:Compound:Gene',
       'bioarx::Covid2_acc_host_gene::Disease:Gene',
       'bioarx::Coronavirus_ass_host_gene::Disease:Gene',
       'DGIDB::INHIBITOR::Gene:Compound',
       'DGIDB::ANTAGONIST::Gene:Compound', 'DGIDB::OTHER::Gene:Compound',
       'DGIDB::AGONIST::Gene:Compound', 'DGIDB::BINDER::Gene:Compound',
       'DGIDB::MODULATOR::Gene:Compound', 'DGIDB::BLOCKER::Gene:Compound',
       'DGIDB::CHANNEL BLOCKER::Gene:Compound',
       'DGIDB::ANTIBODY::Gene:Compound',
       'DGIDB::POSITIVE ALLOSTERIC MODULATOR::Gene:Compound',
       'DGIDB::ALLOSTERIC MODULATOR::Gene:Compound',
       'DGIDB::ACTIVATOR::Gene:Compound',
       'DGIDB::PARTIAL AGONIST::Gene:Compound',
       'DRUGBANK::x-atc::Compound:Atc',
       'DRUGBANK::ddi-interactor-in::Compound:Compound',
       'DRUGBANK::target::Compound:Gene',
       'DRUGBANK::enzyme::Compou

In [15]:
df[df[1].apply(lambda s: s[:8] == 'DRUGBANK')]

,0,1,2
111046,Compound::DB00001,DRUGBANK::x-atc::Compound:Atc,Atc::B01AE02
111047,Compound::DB00001,DRUGBANK::ddi-interactor-in::Compound:Compound,Compound::DB06605
111048,Compound::DB00001,DRUGBANK::ddi-interactor-in::Compound:Compound,Compound::DB06695
111049,Compound::DB00001,DRUGBANK::ddi-interactor-in::Compound:Compound,Compound::DB01254
111050,Compound::DB00001,DRUGBANK::ddi-interactor-in::Compound:Compound,Compound::DB01609
...,...,...,...
1535831,Compound::DB15578,DRUGBANK::x-atc::Compound:Atc,Atc::G
1535832,Compound::DB15598,DRUGBANK::x-atc::Compound:Atc,Atc::B03AB
1535833,Compound::DB15598,DRUGBANK::x-atc::Compound:Atc,Atc::B03A
1535834,Compound::DB15598,DRUGBANK::x-atc::Compound:Atc,Atc::B03


In [16]:
drugbank_smiles = pd.read_csv('../Data/DRKG/drugbank_info/drugbank_smiles.txt', sep='\t', header=None)

In [17]:
drugbank_smiles = drugbank_smiles[1].unique()

In [18]:
len(drugbank_smiles)

8830

In [19]:
 pd.read_csv("../Data/L1000_PhaseI/GSE92742_Broad_LINCS/GSE92742_Broad_LINCS_pert_info.txt", sep="\t",
                        index_col="pert_id")

,pert_iname,pert_type,is_touchstone,inchi_key_prefix,inchi_key,canonical_smiles,pubchem_cid
pert_id,,,,,,,
56582,AKT2,trt_oe,0,-666,-666,-666,-666
5981,HSF1,trt_oe,0,-666,-666,-666,-666
7150,NFE2L2,trt_oe,0,-666,-666,-666,-666
ABL1_G2A,ABL1,trt_oe.mut,0,-666,-666,-666,-666
ABL1_T315I,ABL1,trt_oe.mut,0,-666,-666,-666,-666
...,...,...,...,...,...,...,...
ccsbBroad304_99991,LUCIFERASE,ctl_vector,0,-666,-666,-666,-666
ccsbBroad304_99994,lacZ,ctl_vector,0,-666,-666,-666,-666
ccsbBroad304_99997,eGFP,ctl_vector,0,-666,-666,-666,-666


In [20]:
pert_info_1 = pd.read_csv("../Data/L1000_PhaseI/GSE92742_Broad_LINCS/GSE92742_Broad_LINCS_pert_info.txt", sep="\t",
                        index_col="pert_id", usecols=["pert_id", "pert_iname", "canonical_smiles"])
pert_info_2 = pd.read_csv("../Data/L1000_PhaseII/GSE70138_Broad_LINCS/pert_info_2017-03-06.txt", sep="\t",
                          index_col="pert_id", usecols=["pert_id", "pert_iname", "canonical_smiles"])
pert_info = pd.concat([pert_info_1, pert_info_2], sort=False)

In [21]:
pert_info

,pert_iname,canonical_smiles
pert_id,,
56582,AKT2,-666
5981,HSF1,-666
7150,NFE2L2,-666
ABL1_G2A,ABL1,-666
ABL1_T315I,ABL1,-666
...,...,...
BRDN0000585417,LACZ,-666
BRDN0000562990,EGFP,-666
BRDN0000585533,LUCIFERASE,-666


In [22]:
l1000_smiles = pert_info['canonical_smiles'].unique()

In [23]:
len(l1000_smiles)

21213

In [24]:
len(set(drugbank_smiles).intersection(set(l1000_smiles)))

104

In [25]:
pickle_in = open('../Data/L1000_PhaseI/dataframe.pkl', "rb")
data = pickle.load(pickle_in)

In [26]:
data

,gene_expr
cid,
CPC005_A375_6H:BRD-A85280935-003-01-7:10,"[-0.6806448, -0.52947366, -0.4659655, -1.73020..."
CPC005_A375_6H:BRD-A07824748-001-02-6:10,"[1.8314098, -0.75270295, 0.8809619, -0.2224250..."
CPC004_A375_6H:BRD-K20482099-001-01-1:10,"[3.9644134, 3.3888953, -1.7222486, -0.30714303..."
CPC005_A375_6H:BRD-K62929068-001-03-3:10,"[1.5089136, 0.64932656, -0.6492754, -0.0263244..."
CPC005_A375_6H:BRD-K43405658-001-01-8:10,"[0.8581516, -0.7991509, 0.13817382, -1.2157025..."
...,...
PCLB003_PC3_24H:BRD-A75409952-001-01-6:0.12,"[-0.64300364, -0.9303427, -0.48076797, 0.33226..."
PCLB003_PC3_24H:BRD-A75409952-001-01-6:0.04,"[0.48386264, -1.1165136, -0.4914876, 0.2097616..."
PCLB003_PC3_24H:BRD-K42573370-001-01-1:10,"[-1.8989, 2.9124, 0.0918, 3.4367, 1.5604, -4.1..."


In [27]:
dict_id_to_name = np.load('../Data/DRKG/dict_id_to_name.npy', allow_pickle=True).item()

In [28]:
all_names = []
for value in dict_id_to_name.values():
    all_names.extend(value)

In [29]:
len(all_names)

33168

In [30]:
pert_info['pert_iname'].values

array(['AKT2', 'HSF1', 'NFE2L2', ..., 'LUCIFERASE', 'EGFP', 'EGFP'],
      dtype=object)

In [31]:
all_names

['Lepirudin',
 'Hirudin variant-1',
 'Lepirudin recombinant',
 'DB00001',
 'Cetuximab',
 'DB00002',
 'Dornase alfa',
 'Deoxyribonuclease (human clone 18-1 protein moiety)',
 'Dornase alfa, recombinant',
 'Dornase alpha',
 'Recombinant deoxyribonuclease (DNAse)',
 'DB00003',
 'Denileukin diftitox',
 'Denileukin',
 'Interleukin-2/diptheria toxin fusion protein',
 'DB00004',
 'Etanercept',
 'etanercept-szzs',
 'etanercept-ykro',
 'Recombinant human TNF',
 'rhu TNFR:Fc',
 'rhu-TNFR:Fc',
 'TNFR-Immunoadhesin',
 'DB00005',
 'Peginterferon alfa-2a',
 'PEG-IFN alfa-2A',
 'PEG-Interferon alfa-2A',
 'Pegylated Interfeaon alfa-2A',
 'Pegylated interferon alfa-2a',
 'Pegylated interferon alpha-2a',
 'Pegylated-interferon alfa 2a',
 'DB00008',
 'Alteplase',
 'Alteplase (genetical recombination)',
 'Alteplase, recombinant',
 'Alteplase,recombinant',
 'Plasminogen activator (human tissue-type protein moiety)',
 'rt-PA',
 't-PA',
 't-plasminogen activator',
 'Tissue plasminogen activator',
 'Tissue pl

In [32]:
l1000_names = list(pert_info['pert_iname'].values)

In [33]:
l1000_names = [name.lower() for name in l1000_names]

In [34]:
l1000_names

['akt2',
 'hsf1',
 'nfe2l2',
 'abl1',
 'abl1',
 'ache',
 'acvr1',
 'acvr1',
 'acvr1',
 'acvr1|acvr1|acvr1|acvr1|acvrl1|acvrl1|acvrl1|acvrl1',
 'acvr1|acvr1|acvr1|acvr1|acvrl1|acvrl1|acvrl1|acvrl1',
 'acvr1|acvr1|acvr1|acvr1|acvrl1|acvrl1|acvrl1|acvrl1',
 'acvr1|acvr1|acvr1|acvr1|acvrl1|acvrl1|acvrl1|acvrl1',
 'adm',
 'agt',
 'ahsg',
 'aimp1',
 'akt2',
 'alk',
 'alk',
 'alk',
 'ang',
 'angpt1',
 'angpt2',
 'angpt4',
 'apcs',
 'apln',
 'apoa4',
 'apoe',
 'areg',
 'artn',
 'avp',
 'b2m',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bcr-abl',
 'bdnf',
 'bglap',

In [35]:
drkg_names = [name.lower() for name in all_names]

In [36]:
drkg_names

['lepirudin',
 'hirudin variant-1',
 'lepirudin recombinant',
 'db00001',
 'cetuximab',
 'db00002',
 'dornase alfa',
 'deoxyribonuclease (human clone 18-1 protein moiety)',
 'dornase alfa, recombinant',
 'dornase alpha',
 'recombinant deoxyribonuclease (dnase)',
 'db00003',
 'denileukin diftitox',
 'denileukin',
 'interleukin-2/diptheria toxin fusion protein',
 'db00004',
 'etanercept',
 'etanercept-szzs',
 'etanercept-ykro',
 'recombinant human tnf',
 'rhu tnfr:fc',
 'rhu-tnfr:fc',
 'tnfr-immunoadhesin',
 'db00005',
 'peginterferon alfa-2a',
 'peg-ifn alfa-2a',
 'peg-interferon alfa-2a',
 'pegylated interfeaon alfa-2a',
 'pegylated interferon alfa-2a',
 'pegylated interferon alpha-2a',
 'pegylated-interferon alfa 2a',
 'db00008',
 'alteplase',
 'alteplase (genetical recombination)',
 'alteplase, recombinant',
 'alteplase,recombinant',
 'plasminogen activator (human tissue-type protein moiety)',
 'rt-pa',
 't-pa',
 't-plasminogen activator',
 'tissue plasminogen activator',
 'tissue pl

In [37]:
len(set(drkg_names).intersection(set(l1000_names)))

1730

In [38]:
len(l1000_names)

53553

## Comparing fingerprints with Genevieve's code

In [39]:
drugbank_smiles

array(['CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)[C@H](CC(O)=O)NC(=O)CNC(=O)[C@H](CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=O)[C@@H]1CCCN1C(=O)[C@H](N)CC1=CC=CC=C1)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CC1=CC=C(O)C=C1)C(=O)N[C@@H](CC(C)C)C(O)=O',
       'CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=O)[C@H](CC(C)C)NC(=O)[C@@H](CC(C)C)NC(=O)[C@H](CC1=CC=C(O)C=C1)NC(=O)[C@H](CO)NC(=O)[C@H](CC1=CNC2=C1C=CC=C2)NC(=O)[C@H](CC1=CNC=N1)NC(=O)[C@@H]1CCC(=O)N1',
       'CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H](CC1=CC=C(O)C=C1)NC(=O)[C@H](CO)NC(=O)[C@H](CC1=CNC2=CC=CC=C12)NC(=O)[C@H](CC1=CN=CN1)NC(=O)[C@@H]1CCC(=O)N1)C(=O)N[C@@H](CCCN=C(N)N)C(=O)N1CCC[C@H]1C(=O)NNC(N)=O',
       ..., 'CC(C)N(CC[C@H](C1=CC=CC=C1)C1=CC(CO)=CC=C1O)C(C)C',
       '[Na+].CCOC1=CC=C(NC2=CC=C(C=C2)C(=C2\\C=C/C(/C=C2C)=[N+](/CC)CC2=CC(=CC=C2)S([O-])(=O)=O)\\C2=C(C)C=C(C=C2)N(CC)CC2=CC(=CC=C2)S(

In [40]:
l1000_smiles

array(['-666', 'CC1CS(=O)(=O)CCN1N=Cc1ccc(o1)[N+]([O-])=O',
       'NC(Cc1c[nH]c2cccc(O)c12)C(O)=O', ...,
       'CCCc1cc2c(ncnc2s1)N1CCN(CC1)C1=NCC(C)(C)S1',
       'COc1cc(O)c2c(c1)oc(cc2=O)-c1ccc(OC)c(c1)-c1c(O)cc(O)c2c1oc(cc2=O)-c1ccc(O)cc1',
       'O=C1c2ccccc2-c2nc3nonc3nc12'], dtype=object)

In [41]:
len(drugbank_smiles)

8830

In [42]:
len(l1000_smiles)

21213

In [44]:
molecules_db = [Chem.MolFromSmiles(smile) for smile in drugbank_smiles]
molecules_l1000 = [Chem.MolFromSmiles(smile) for smile in l1000_smiles]

RDKit ERROR: [14:37:59] Explicit valence for atom # 2 O, 3, is greater than permitted
RDKit ERROR: [14:37:59] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [14:37:59] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [14:37:59] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [14:37:59] Explicit valence for atom # 13 Cl, 5, is greater than permitted
RDKit ERROR: [14:37:59] Explicit valence for atom # 14 N, 5, is greater than permitted
RDKit ERROR: [14:37:59] Explicit valence for atom # 19 O, 3, is greater than permitted
RDKit ERROR: [14:37:59] Explicit valence for atom # 2 O, 3, is greater than permitted
RDKit ERROR: [14:37:59] Explicit valence for atom # 6 N, 4, is greater than permitted
RDKit ERROR: [14:37:59] Explicit valence for atom # 11 N, 4, is greater than permitted
RDKit ERROR: [14:37:59] Explicit valence for atom # 0 O, 3, is greater than permitted
RDKit ERROR: [14:37:59] Explicit valence for atom

In [47]:
print(len(drugbank_smiles), len(molecules_db))
print(len(l1000_smiles), len(molecules_l1000))

8830 8830
21213 21213


In [48]:
# Remove None values
molecules_db = [mol for mol in molecules_db if mol is not None]
molecules_l1000 = [mol for mol in molecules_l1000 if mol is not None]

In [49]:
print(len(drugbank_smiles), len(molecules_db))
print(len(l1000_smiles), len(molecules_l1000))

8830 8806
21213 21211


In [50]:
recomputed_smiles_db = [Chem.MolToSmiles(mol) for mol in molecules_db]
recomputed_smiles_l1000 = [Chem.MolToSmiles(mol) for mol in molecules_l1000]

In [51]:
len(set(recomputed_smiles_db).intersection(set(recomputed_smiles_l1000)))

1574

In [52]:
hashstring_db = [rdMolHash.GenerateMoleculeHashString(mol) for mol in molecules_db]
hashstring_l1000 = [rdMolHash.GenerateMoleculeHashString(mol) for mol in molecules_l1000]

RDKit WARNING: [14:44:53] Warning: atom with more than 1 ring-stereo atoms found.
RDKit WARNING: [14:44:53] Warning: atom with more than 1 ring-stereo atoms found.
RDKit WARNING: [14:44:53] Warning: atom with more than 1 ring-stereo atoms found.
RDKit WARNING: [14:44:53] Warning: atom with more than 1 ring-stereo atoms found.
RDKit WARNING: [14:44:53] Warning: atom with more than 1 ring-stereo atoms found.
RDKit WARNING: [14:44:53] Warning: atom with more than 1 ring-stereo atoms found.
RDKit WARNING: [14:44:53] Warning: atom with more than 1 ring-stereo atoms found.
RDKit WARNING: [14:44:53] Warning: atom with more than 1 ring-stereo atoms found.
RDKit WARNING: [14:44:53] Warning: atom with more than 1 ring-stereo atoms found.
RDKit WARNING: [14:44:53] Warning: atom with more than 1 ring-stereo atoms found.
RDKit WARNING: [14:44:53] Warning: atom with more than 1 ring-stereo atoms found.
RDKit WARNING: [14:44:53] Warning: atom with more than 1 ring-stereo atoms found.
RDKit WARNING: [

In [56]:
len(set(hashstring_db).intersection(set(hashstring_l1000)))

1610